<a href="https://colab.research.google.com/github/giacomogreggio/Algoritmi_avanzati/blob/master/Alg_Av_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from math import pi, cos, acos, sqrt, inf
import time
from google.colab import drive
import random
import os
import matplotlib.pylab as plt
%matplotlib inline
from statistics import mean
import copy

In [2]:
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
dataset_folder="/content/drive/My Drive/Colab Notebooks/algoritmi_avanzati/Homework_2/dataset/ulysses22.tsp"

In [0]:
RRR = 6378.388

def convert_to_rad(coords):
    res = []
    for c in coords:
        deg = int(c)
        m = c - deg
        rad = pi * (deg + 5.0 * m/ 3.0) / 180.0
        res.append(rad)
    return res
 
def geographic_distance(coord1, coord2):
    q1 = cos( coord1[1] - coord2[1] )
    q2 = cos( coord1[0] - coord2[0] )
    q3 = cos( coord1[0] + coord2[0] )
    dij = int( RRR * acos( 0.5*((1.0+q1)*q2 - (1.0-q1)*q3) ) + 1.0)
    return dij

def euclidean_distance(coord1, coord2):
    return sqrt(sum([(a - b) ** 2 for a, b in zip(coord1, coord2)]))

def import_graph(dataset_path):
    dataset_file=open(dataset_path,"r")
    content=dataset_file.read()
    dataset=content.split("\n")
    i = 0
    while not dataset[i].startswith("NODE_COORD_SECTION"):
        if dataset[i].startswith("EDGE_WEIGHT_TYPE"):
            type_coords = dataset[i].split(":")[1][1:]
        i = i + 1
    i = i + 1

    node_names, coordinates = [], {}
    for j in range(i, len(dataset)):
        row = dataset[j].split()
        if len(row) != 3:
            break
        name = str(row[0])
        x, y = float(row[1]), float(row[2])
        node_names.append(name)
        coordinates[name] = [x, y] if type_coords == "EUC_2D" else convert_to_rad([x, y])
    return node_names, coordinates, euclidean_distance if type_coords == "EUC_2D" else geographic_distance


In [102]:
node_names, coordinates, distance_function = import_graph(dataset_folder)
d, p = {}, {}
starting_node = node_names[0]

def get_name(v, s):
    return ''.join(s) + " " + str(v)

def held_karp(v, S):
    if v not in S:
        print("Error")
    # ----------------------------------------------------------------TODO scegliere il nome----------------------------------------------------------------
    name_to_choose = get_name(v, S) 
    if len(S) == 1 and v in S:
        return distance_function(coordinates[starting_node], coordinates[v])
    elif name_to_choose in d:
        return d[name_to_choose]
    else:
        mindist, minprec = inf, None
        node_names_copy = copy.deepcopy(S)
        node_names_copy.remove(v)
        for u in node_names_copy:
            dist = held_karp(u, node_names_copy)
            uv_weight = distance_function(coordinates[u], coordinates[v])
            if dist + uv_weight < mindist:
                mindist = dist + uv_weight
                minprec = u
        d[name_to_choose] = mindist
        p[name_to_choose] = minprec
        return mindist

held_karp(starting_node, node_names)



7013